# Data Science Project SoSe 2024
## Team 07
- Maximilian Hoffmann
- Kilian Kempf
- Daniel Schneider
- Tom Schuck

## Submission of Task 2

### Libraries 

In [61]:
import os

import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier

from user_ts_cv_splitter import LastUserTSCVSplitter

### Data Preparation

In [62]:
DATA_DIR = os.path.join(os.getcwd(), 'data/Instacart')

op_prior = pd.read_csv(os.path.join(DATA_DIR, 'order_products__prior.csv.zip'))
op_train = pd.read_csv(os.path.join(DATA_DIR, 'order_products__train.csv.zip'))
op = pd.concat([op_prior, op_train])

tip_train = pd.read_csv(os.path.join(DATA_DIR, 'tip_trainingsdaten1_.csv'))[['order_id', 'tip']]
tip_test = pd.read_csv(os.path.join(DATA_DIR, 'tip_testdaten1_template.csv'))
tip = pd.concat(
    [tip_train, tip_test[['order_id', 'tip']]])

orders = pd.read_csv(os.path.join(DATA_DIR, 'orders.csv.zip'))
orders_tip = pd.merge(orders, tip)
orders_tip

order_id  user_id eval_set  order_number  order_dow  \
0         2539329        1    prior             1          2   
1         2398795        1    prior             2          3   
2          473747        1    prior             3          3   
3         2254736        1    prior             4          4   
4          431534        1    prior             5          4   
...           ...      ...      ...           ...        ...   
3346078   2266710   206209    prior            10          5   
3346079   1854736   206209    prior            11          4   
3346080    626363   206209    prior            12          1   
3346081   2977660   206209    prior            13          1   
3346082    272231   206209    train            14          6   

         order_hour_of_day  days_since_prior_order  tip  
0                        8                     NaN  0.0  
1                        7                    15.0  0.0  
2                       12                    21.0  0.0  
3                        7                    29.0  0.0  
4                       15                    28.0  0.0  
...                    ...                     ...  ...  
3346078                 18                    29.0  0.0  
3346079                 10                    30.0  0.0  
3346080                 12                    18.0  0.0  
3346081                 12                     7.0  0.0  
3346082                 14                    30.0  NaN  

[3346083 rows x 8 columns]

### Feature Engineering

In [63]:
# NaN values in the 'days_since_prior_order' column (first orders) are replaced by -1, to ensure that the training of the decision tree classifier is more robust. 
orders_tip['days_since_prior_order'] = orders_tip['days_since_prior_order'].fillna(-1).astype(int)

# The tip history, which represents the ratio of orders for which the user has tipped in the past, is calculated as a new feature. We assume that the encoded tip history may be an important indicator of future tipping behavior.
orders_tip['tip'] = orders_tip['tip'].astype(bool)
orders_tip['tip_history'] = orders_tip.groupby('user_id')['tip'].transform('cumsum').shift(1) / orders_tip[
    'order_number'].shift(1)

# NaN values in the 'tip_history' column (first orders) are replaced by -1, to ensure that the training of the decision tree classifier is more robust.
orders_tip.loc[orders_tip['order_number'] == 1, 'tip_history'] = -1

# Test orders, for which no tip is provided, are identified and the 'tip' column is set to NaN (previous conversion to boolean assigned True as value to these orders)
orders_tip['tip'] = orders_tip['tip'].astype(object)
orders_tip.loc[orders_tip['order_id'].isin(tip_test['order_id']), 'tip'] = np.nan
orders_tip

order_id  user_id eval_set  order_number  order_dow  \
0         2539329        1    prior             1          2   
1         2398795        1    prior             2          3   
2          473747        1    prior             3          3   
3         2254736        1    prior             4          4   
4          431534        1    prior             5          4   
...           ...      ...      ...           ...        ...   
3346078   2266710   206209    prior            10          5   
3346079   1854736   206209    prior            11          4   
3346080    626363   206209    prior            12          1   
3346081   2977660   206209    prior            13          1   
3346082    272231   206209    train            14          6   

         order_hour_of_day  days_since_prior_order    tip  tip_history  
0                        8                      -1  False    -1.000000  
1                        7                      15  False     0.000000  
2                       12                      21  False     0.000000  
3                        7                      29  False     0.000000  
4                       15                      28  False     0.000000  
...                    ...                     ...    ...          ...  
3346078                 18                      29  False     0.222222  
3346079                 10                      30  False     0.200000  
3346080                 12                      18  False     0.181818  
3346081                 12                       7  False     0.166667  
3346082                 14                      30    NaN     0.153846  

[3346083 rows x 9 columns]

### Task 2.1

In [64]:
orders_tip_train = pd.merge(orders_tip, tip_train)
orders_tip_train['tip'] = orders_tip_train['tip'].astype(bool)
orders_tip_train

order_id  user_id eval_set  order_number  order_dow  \
0         2539329        1    prior             1          2   
1         2398795        1    prior             2          3   
2          473747        1    prior             3          3   
3         2254736        1    prior             4          4   
4          431534        1    prior             5          4   
...           ...      ...      ...           ...        ...   
3214869   2558525   206209    prior             9          4   
3214870   2266710   206209    prior            10          5   
3214871   1854736   206209    prior            11          4   
3214872    626363   206209    prior            12          1   
3214873   2977660   206209    prior            13          1   

         order_hour_of_day  days_since_prior_order    tip  tip_history  
0                        8                      -1  False    -1.000000  
1                        7                      15  False     0.000000  
2                       12                      21  False     0.000000  
3                        7                      29  False     0.000000  
4                       15                      28  False     0.000000  
...                    ...                     ...    ...          ...  
3214869                 15                      22  False     0.250000  
3214870                 18                      29  False     0.222222  
3214871                 10                      30  False     0.200000  
3214872                 12                      18  False     0.181818  
3214873                 12                       7  False     0.166667  

[3214874 rows x 9 columns]

In [65]:
# To train the decision tree classifier, the following features are used (unique identifiers are excluded):
features = ['order_number', 'order_dow', 'order_hour_of_day', 'days_since_prior_order', 'tip_history']

# Hyperparameter grid for the decision tree classifier to choose the optimal hyperparameters to train the model
param_grid = {'criterion': ['entropy'],
              'max_depth': [depth for depth in range(1, 21)],
              'min_samples_leaf': [2 ** i for i in range(0, 15)]}

# Multiple cross-validation strategies were tested to find the most suitable one. Strategies which were tested, but not used for the final training of the model, are commented out.

# Standard cross-validation (not suitable for time-series-like data)
# cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Custom user time-series aware cross-validation, which selects a percentage (in this case 10%) of last orders per user as a validation set and the remaining orders as a training set. This is repeated for multiple splits (in this case 5). The selected validation set of a split is completely left out in the following splits.

# cv_user_ts = UserTSCVSplitter(n_splits=5, validation_set_ratio=0.1, orders_by_user=orders_tip_train[['user_id']])

# Custom user time-series aware cross-validation, which selects the last order per user as a validation set and the remaining orders as a training set. This is repeated for multiple splits (in this case 5). The selected validation set of a split is completely left out in the following splits.
# This strategy was chosen for the final training of the model, because it is best suited for the given data and the task to predict the tip of the next order placed by a user.
cv_last_user_ts = LastUserTSCVSplitter(n_splits=5, orders_by_user=orders_tip_train[['user_id', 'order_number']])

grid_search_clf = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=cv_last_user_ts, n_jobs=-1, scoring='accuracy')

In [66]:
X = orders_tip_train[features]
y = orders_tip_train[['tip']]
grid_search_clf.fit(X, y)

GridSearchCV(cv=LastUserTSCVSplitter(n_splits=5,
                                     orders_by_user=         user_id  order_number
0              1             1
1              1             2
2              1             3
3              1             4
4              1             5
...          ...           ...
3214869   206209             9
3214870   206209            10
3214871   206209            11
3214872   206209            12
3214873   206209            13

[3214874 rows x 2 columns]),
             estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['entropy'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19, 20],
                         'min_samples_leaf': [1, 2, 4, 8, 16, 32, 64, 128, 256,
                                              512, 1024, 2048, 4096, 8192,
                                              16384]},
             scoring='accuracy')

In [67]:
decision_tree_clf = grid_search_clf.best_estimator_
grid_search_clf.best_params_

{'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 256}

### Task 2.2

##### Estimated accuracy of the decision tree classifier on the test set:

In [68]:
grid_search_clf.best_score_

0.7839058963532112

### Task 2.3

In [69]:
# The following trivial strategy is used to predict the most frequent class (majority class) of the training set.
most_freq_clf = DummyClassifier(strategy='most_frequent')
most_freq_score = cross_val_score(most_freq_clf, X, y, cv=cv_last_user_ts, scoring='accuracy').mean()
most_freq_score

0.5591559729508828

##### Benefit of decision tree classifier over classifier that predicts most frequent class:

In [70]:
grid_search_clf.best_score_ - most_freq_score

0.22474992340232836

### Task 2.4

In [71]:
orders_tip_test = pd.merge(orders_tip.drop('tip', axis=1), tip_test)
orders_tip_test['tip'] = decision_tree_clf.predict(orders_tip_test[features])
orders_tip_test

order_id  user_id eval_set  order_number  order_dow  \
0        1187899        1    train            11          4   
1        1492625        2    train            15          1   
2        2196797        5    train             5          0   
3         525192        7    train            21          2   
4         880375        8    train             4          1   
...          ...      ...      ...           ...        ...   
131204   2585586   206199    train            20          2   
131205    943915   206200    train            24          6   
131206   2371631   206203    train             6          4   
131207   1716008   206205    train             4          1   
131208    272231   206209    train            14          6   

        order_hour_of_day  days_since_prior_order  tip_history  Unnamed: 0  \
0                       8                      14     0.000000          10   
1                      11                      30     0.571429          25   
2                      11                       6     0.500000          49   
3                      11                       6     0.650000          74   
4                      14                      10     0.666667          78   
...                   ...                     ...          ...         ...   
131204                 16                      30     1.000000     3420838   
131205                 19                       6     0.652174     3420862   
131206                 19                      30     0.000000     3420924   
131207                 16                      10     0.000000     3420933   
131208                 14                      30     0.153846     3421082   

          tip  
0       False  
1        True  
2        True  
3        True  
4        True  
...       ...  
131204   True  
131205   True  
131206  False  
131207  False  
131208  False  

[131209 rows x 10 columns]

In [72]:
orders_tip_test_csv = orders_tip_test[tip_test.columns].copy()
orders_tip_test_csv.rename(columns={tip_test.columns[0]: ''}, inplace=True)
orders_tip_test_csv

order_id    tip
0            10   1187899  False
1            25   1492625   True
2            49   2196797   True
3            74    525192   True
4            78    880375   True
...         ...       ...    ...
131204  3420838   2585586   True
131205  3420862    943915   True
131206  3420924   2371631  False
131207  3420933   1716008  False
131208  3421082    272231  False

[131209 rows x 3 columns]

In [73]:
orders_tip_test_csv.to_csv(os.path.join(DATA_DIR, 'tip_testdaten1.csv'), index=False)

### Task 2.5 a)

In [74]:
first_orders = orders[orders['order_number'] == 1]
orders_21137 = op[op['product_id'] == 21137]
first_orders_21137 = first_orders[first_orders['order_id'].isin(orders_21137['order_id'])]
first_orders_21137

order_id  user_id eval_set  order_number  order_dow  \
1159      2669994       77    prior             1          2   
1518       185725      101    prior             1          2   
1598      2313085      108    prior             1          0   
1823      1747819      126    prior             1          6   
1943      3204695      135    prior             1          3   
...           ...      ...      ...           ...        ...   
3420524   2962368   206178    prior             1          3   
3420593     31169   206183    prior             1          0   
3420750   3395125   206194    prior             1          1   
3420925   1438269   206204    prior             1          1   
3420930    969311   206205    prior             1          4   

         order_hour_of_day  days_since_prior_order  
1159                    11                     NaN  
1518                    11                     NaN  
1598                    13                     NaN  
1823                    11                     NaN  
1943                    19                     NaN  
...                    ...                     ...  
3420524                 10                     NaN  
3420593                 13                     NaN  
3420750                  9                     NaN  
3420925                 11                     NaN  
3420930                 12                     NaN  

[16464 rows x 7 columns]

##### Percentage of orders containing product 21137 (organic strawberries) first orders of the respective user:

In [75]:
(first_orders_21137.shape[0] / first_orders.shape[0]) * 100

7.984132603329631

### Task 2.5 b)

In [76]:
last_orders = orders[['user_id', 'order_number']].groupby('user_id').max().reset_index()
last_orders = pd.merge(orders, last_orders, on=['user_id', 'order_number'])
last_orders_21137 = last_orders[last_orders['order_id'].isin(orders_21137['order_id'])]
last_orders_21137

order_id  user_id eval_set  order_number  order_dow  \
17       2461523       18    train             7          6   
26       2614670       27    train            82          5   
58       1651215       59    train            11          0   
62       2989905       63    train            40          6   
78         21708       79    train             8          0   
...          ...      ...      ...           ...        ...   
206152    642502   206153    train            14          0   
206173   1620607   206174    train            53          4   
206187   3367945   206188    train             8          1   
206199    943915   206200    train            24          6   
206204   1716008   206205    train             4          1   

        order_hour_of_day  days_since_prior_order  
17                      9                     7.0  
26                     14                     4.0  
58                     12                    10.0  
62                      8                    13.0  
78                      6                    30.0  
...                   ...                     ...  
206152                  8                    21.0  
206173                 22                    10.0  
206187                 21                    30.0  
206199                 19                     6.0  
206204                 16                    10.0  

[10894 rows x 7 columns]

##### Percentage of orders containing product 21137 (organic strawberries) last orders of the respective user:

In [77]:
(last_orders_21137.shape[0] / last_orders.shape[0]) * 100

5.282989588233297

##### Comparison:
Percentage of first orders containing product 21137 (organic strawberries) of the respective user: $\approx 7.984\%$\
Percentage of last orders containing product 21137 (organic strawberries) of the respective user: $\approx 5.283\%$

Therefore, the ratio of first orders containing product 21137 (organic strawberries) is $\approx 51.126\%$ larger than the ratio of last orders containing this product. This seems to be rather a systematic reason than being random due to the relatively large difference. It seems that users buy the product 21137 (organic strawberries) less often in their last order compared to their first order.
This can be explained by a variety of possible reasons, such as:
- Users may have tried the product and did not like it. Therefore, they do not buy the product again, which is why the product is less likely to be included in the last order.
- The product may have been discontinued or is not available anymore. If the product was available in the past, but was discontinued, it is more likely that the product is included in the first order than in the last order, because orders which are placed after the discontinuation of the product cannot contain the product anymore.